In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 使 GPU 0 和 1 可见


import sys
sys.path.append("..")
sys.path.append("../mango")
sys.path.append("../mango/design_baselines/diff")


import numpy as np
import torch
import pytorch_lightning as pl

import mango_jupyter_scripts
import design_bench
from helpers import seed_everything 

from SDE import DiffusionSOO



wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
import argparse
import logging

def build_args() -> argparse.Namespace:
    """
    Defines the experimental arguments for offline MBO baseline method
    evaluation.
    Input:
        None.
    Returns:
        A namespace containing the experimental argument values.
    """
    parser = argparse.ArgumentParser(description="Our method Experiments")

    parser.add_argument(
        "--task",
        type=str,
        default='TFBind8-Exact-v0', 
        choices=[task.task_name for task in design_bench.registry.all()],
        help="The name of the design-bench task for the experiment."
    )
    parser.add_argument(
        "--logging-dir",
        type=str,
        default=None,
        help="Logging directory to save optimization results to."
    )
    parser.add_argument(
        "--budget", type=int, default=2048, help="Query budget. Default 2048."
    )
    parser.add_argument(
        "--hidden-size",
        type=int,
        default=2048,
        help="Hidden size of the surrogate objective model. Default 2048."
    )
    parser.add_argument(
        "--seed", type=int, default=2024, help="Random seed. Default 42."
    )
    parser.add_argument(
        "--device", type=str, default="auto", help="Device. Default `auto`."
    )
    args = parser.parse_args([])
    return args

args = build_args()

In [ ]:
from mango_jupyter_scripts.training import train_surrogate, mango_train
# Training a surrogate forward model (It is merely to provide a forward model for comparison purposes only).
forwardmodel = train_surrogate(task_name=args.task, hidden_size=args.hidden_size)

Loading model from ../checkpoints/surrogate/TFBind8-Exact-v0-surrogate-100epoch.ckpt


In [6]:
dataset_kwargs=dict(
    max_samples=10000,
    distribution=None,
    max_percentile=50,
    min_percentile=0) 

In [7]:
task = design_bench.make( args.task, dataset_kwargs=dataset_kwargs)
normalize_x = False
normalize_y = False
x_min, x_max, y_min, y_max = np.min(task.dataset.x, axis=0), np.max(task.dataset.x, axis=0), np.min(task.dataset.y, axis=0), np.max(task.dataset.y, axis=0) 
try:
    x1_range = task.dataset.x1_range
    x2_range = task.dataset.x2_range
    x_lbound = np.array([x1_range[0], x2_range[0]])
    x_ubound = np.array([x1_range[1], x2_range[1]])
except:
    x_lbound = task.oracle.internal_dataset.x.min(axis=0)
    x_ubound = task.oracle.internal_dataset.x.max(axis=0)
    y_lbound = task.oracle.internal_dataset.y.min()
    y_ubound = task.oracle.internal_dataset.y.max()
    
print("##################Before Normalization##################")
print('x_min:', x_min, '\n', 'x_max:',x_max, '\n','y_min:',y_min,'\n', 'y_max:',y_max)
print('x_lbound:', x_lbound, '\n', 'x_ubound:',x_ubound, '\n','y_lbound:',y_lbound,'\n', 'y_ubound:',y_ubound)
print('x.shape:', task.x.shape, '\n', 'y.shape:',task.y.shape)

if normalize_y:
    task.map_normalize_y()
    y_min = task.normalize_y(y_min)[0]
    y_max = task.normalize_y(y_max)[0]
    y_lbound = task.normalize_y(y_lbound)[0]
    y_ubound = task.normalize_y(y_ubound)[0]
if normalize_x:
    task.map_normalize_x()
    x_min = task.normalize_x(x_min)[0]
    x_max = task.normalize_x(x_max)[0]
    x_lbound = task.normalize_x(x_lbound)[0]
    x_ubound = task.normalize_x(x_ubound)[0]

if normalize_x or normalize_y:
    print("##################After Normalization##################")
    print('x_min:', x_min, '\n', 'x_max:',x_max, '\n','y_min:',y_min,'\n', 'y_max:',y_max)
    print('x_lbound:', x_lbound, '\n', 'x_ubound:',x_ubound, '\n','y_lbound:',y_lbound,'\n', 'y_ubound:',y_ubound)
    print('x.shape:', task.x.shape, '\n', 'y.shape:',task.y.shape)

##################Before Normalization##################
x_min: [0 0 0 0 0 0 0 0] 
 x_max: [3 3 3 3 3 3 3 3] 
 y_min: [0.] 
 y_max: [0.43929616]
x_lbound: [0 0 0 0 0 0 0 0] 
 x_ubound: [3 3 3 3 3 3 3 3] 
 y_lbound: 0.0 
 y_ubound: 1.0
x.shape: (10000, 8) 
 y.shape: (10000, 1)


In [ ]:
import shutil
args = build_args()

seed_everything(args.seed)
torch.set_default_dtype(torch.float32)
import warnings
warnings.filterwarnings("ignore", message="The dataloader, .* does not have many workers.*")

try:
    # # 尝试访问 inverse_model
    inverse_model

except NameError:
    inverse_model = None  # 如果未定义，设置为 None

# inverse_model = None
num_epochs=  800

args.logging_dir = args.logging_dir or f"../logs/run_ours_{args.task}_seed{args.seed}_budget{args.budget}_hidden{args.hidden_size}_epoch{num_epochs}_betamax{0.05}_nonormalizedY{normalize_y}"
print(args.logging_dir )

# logging_dir = args.logging_dir
# while os.path.exists(logging_dir):
#     logging_dir = f"{args.logging_dir}_v{version}"
#     version += 1
# os.makedirs(logging_dir, exist_ok=True)


# inverse_model = mango_train(
#     task=task,
#     task_name=args.task,
#     ckpt_dir=args.logging_dir,
#     hidden_size=args.hidden_size,
#     device=args.device,
#     seed=args.seed,
#     condition_training=False,
#     num_epochs=num_epochs,
#     dropout=0.0,
#     augment=True,
#     lr=5e-5,
#     batch_size=64,
#     debais=True,
# )



Global seed set to 2024


../logs/run_ours_TFBind8-Exact-v0_seed2024_budget2048_hidden2048_epoch800_betamax0.05_nonormalizedYFalse


In [ ]:
if task.is_discrete:
    task.map_to_logits()
    
model_ckpt = f"/home/tzhouaq/ManGO_SOO/logs/run_ours_TFBind8-Exact-v0_seed2024_budget2048_hidden2048_epoch800_betamax0.05_nonormalizedYFalse/ours-tf_bind_8-SIX6_REF_R1/tf_bind_8-seed2024-hidden2048-score_matchingFalse.ckpt"
inverse_model = DiffusionSOO.load_from_checkpoint(
    model_ckpt,
    taskname=task.dataset_name,
    task=task,
    ckpt_dir=args.logging_dir,
    hidden_size=args.hidden_size,
    device=args.device,
    seed=args.seed,
    condition_training=False,
    num_epochs=num_epochs,
    dropout=0.0,
    augment=True,
    lr=5e-5,
    batch_size=64,
    debais=True,
)

uncondition_training T: {1.0}


In [ ]:
from mango_jupyter_scripts.eval_fks_discrete import mango_eval
# This cell is to conduct unconditional generation

augment = True
condition_training = False
if task.is_discrete:
    x_lbound = task.x.reshape(-1, task.x.shape[-1]*task.x.shape[-2]).min(axis=0)
    x_ubound = task.x.reshape(-1, task.x.shape[-1]*task.x.shape[-2]).max(axis=0)
    
if augment:

    clip_min = torch.tensor(np.concatenate((x_lbound, [y_lbound])))
    clip_max = torch.tensor(np.concatenate((x_ubound, [y_ubound])))
else:
    clip_min = torch.tensor(x_min)
    clip_max = torch.tensor(x_max)
 
clip_dic = {'simple_clip': True, 'clip_min': clip_min , 'clip_max': clip_max}
print(clip_dic)

num_samples = args.budget
 
res = mango_eval(
    task=task,
    task_name=args.task,
    forwardmodel=forwardmodel,
    inverse_model=inverse_model,
    device='auto',
    logging_dir=args.logging_dir,
    num_samples=num_samples,
    augment=augment,
    clip_dic=clip_dic,
    condition_training=condition_training,
    num_steps=500,
    seed=args.seed,
    is_discrete=task.is_discrete,
)
 

{'simple_clip': True, 'clip_min': tensor([-1.9459, -1.9459, -1.9459, -1.9459, -1.9459, -1.9459, -1.9459, -1.9459,
        -1.9459, -1.9459, -1.9459, -1.9459, -1.9459, -1.9459, -1.9459, -1.9459,
        -1.9459, -1.9459, -1.9459, -1.9459, -1.9459, -1.9459, -1.9459, -1.9459,
         0.0000]), 'clip_max': tensor([1.9459, 1.9459, 1.9459, 1.9459, 1.9459, 1.9459, 1.9459, 1.9459, 1.9459,
        1.9459, 1.9459, 1.9459, 1.9459, 1.9459, 1.9459, 1.9459, 1.9459, 1.9459,
        1.9459, 1.9459, 1.9459, 1.9459, 1.9459, 1.9459, 1.0000])}
the first 5 (no rank) design (normalized_x=False, normalized_y=False) is:
 tensor([[ 1.0000e+00, -1.0000e+00, -1.0000e+00, -3.2907e-01, -1.7788e+00,
          1.2413e+00,  1.9207e+00, -6.0535e-02, -3.5845e-02,  8.0645e-03,
          1.9446e+00, -1.1530e-02, -3.1601e-02,  1.8888e+00,  3.7332e-02,
         -1.7993e+00, -1.8482e+00, -1.9169e+00,  1.3287e-04, -2.1459e-02,
          1.8623e+00,  1.9282e+00, -2.3346e-02,  2.3155e-02,  4.2208e-01]])


/home/tzhouaq/anaconda3/envs/design-bench/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


In [ ]:
import numpy as np

y_unob_max = task.oracle.internal_dataset.y.max()
y_unob_min = task.oracle.internal_dataset.y.min()

designs = res['x']
preds =   res['y_gen']
# preds = res['y']
scores = res['y_scores']

print("Input structure:\n The number of the designs is: ", designs.shape[1], "; the dimension of the designs is: ", designs.shape[-1])
print("Output structure:\n The number of the predictions is: ", preds.shape[1], "; the dimension of the predictions is: ", preds.shape[-1])

  
# task = design_bench.make(args.task)
if task.is_normalized_y:
    best_training_score = task.denormalize_y(np.max(task.dataset.y))[0]
else:
    best_training_score = np.max(task.dataset.y)
print("Training dataset:\n The best score during training is: ",best_training_score )
oracle = np.max(scores)
print(f'Oracle results:\n On the given design: the Oracle is: {oracle}')
print(f' The corresponding design is: {designs[np.argmax(scores)]}')

k = 1
percentile = 100
one_shot = scores[np.argmax(preds)][0]
normalize_one_shot = (one_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', one_shot, 'the abs gap is: ', np.abs(oracle - one_shot))
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_one_shot, 'the abs gap is: ', np.abs(oracle - one_shot))
 

k = 16
percentile = 100
preds_flat = preds.flatten()
sorted_indices = np.argsort(preds_flat)
top_64_indices = sorted_indices[-k:][::-1]
few_shot = np.percentile(scores[top_64_indices],percentile)
indices = np.where(scores[top_64_indices] == few_shot)[0]
normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f' The corresponding design is: {designs[top_64_indices][indices]}')

k = 128
percentile = 100
preds_flat = preds.flatten()
sorted_indices = np.argsort(preds_flat)
top_64_indices = sorted_indices[-k:][::-1]
few_shot = np.percentile(scores[top_64_indices],percentile)
indices = np.where(scores[top_64_indices] == few_shot)[0]
normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f' The corresponding design is: {designs[top_64_indices][indices]}')
 

k = 256
percentile = 100
preds_flat = preds.flatten()
sorted_indices = np.argsort(preds_flat)
top_64_indices = sorted_indices[-k:][::-1]
few_shot = np.percentile(scores[top_64_indices],percentile)
indices = np.where(scores[top_64_indices] == few_shot)[0]
normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
print(f' The corresponding design is: {designs[top_64_indices][indices]}')

Input structure:
 The number of the designs is:  8 ; the dimension of the designs is:  3
Output structure:
 The number of the predictions is:  1 ; the dimension of the predictions is:  1
Training dataset:
 The best score during training is:  0.43929616
Oracle results:
 On the given design: the Oracle is: 0.9909186363220215
 The corresponding design is: [[ 0.          1.          0.        ]
 [ 0.02881238  1.6316034  -0.00803198]
 [ 0.02410268  1.925548    0.01202452]
 [ 0.01577527  1.8477434  -0.00406471]
 [ 0.01358786  0.00242611  1.8480284 ]
 [-1.9459102  -1.9298172  -0.09024132]
 [-0.03568896 -0.01494862  1.8518175 ]
 [ 1.8733723   0.016536   -0.03814793]]
Evaluation results 3:
 1-shot 100th percentile of the predictions based on oracle score is:  0.8883643 the abs gap is:  0.10255432
Evaluation results 3:
 1-shot 100th percentile of the predictions based on normalized oracle score is:  0.8883643 the abs gap is:  0.10255432
Evaluation results 3:
 16-shot 100th percentile of the pred

In [ ]:
# This cell is to conduct  conditional generation
augment = True
if normalize_y:
    print('y_given before normalization:',  task.oracle.internal_dataset.y.max())
    y_given = torch.tensor(task.normalize_y(task.oracle.internal_dataset.y.max())[0])
    # y_given = torch.tensor(task.oracle.external_dataset.y.max()) 
else:
    y_given = torch.tensor(task.oracle.internal_dataset.y.max())
print('y_given:', y_given)
if y_max.item() < y_given:
    y_max = y_given 
if  y_min.item() > y_given:
    y_min = y_given

if task.is_discrete:
    x_lbound = task.x.reshape(-1, task.x.shape[-1]*task.x.shape[-2]).min(axis=0)
    x_ubound = task.x.reshape(-1, task.x.shape[-1]*task.x.shape[-2]).max(axis=0)

 
clip_dic = {'simple_clip': True, 'clip_min': clip_min , 'clip_max': clip_max}
print(clip_dic)


args.budget = 256
seed_list = [49, 58, 75, 81, 87, 106, 130, 131]
reported_results_1shot = []
reported_results_16shot = []
reported_results_128shot = []
reported_results_256shot = []

args.budget = 256
num_steps = 250
inference_scaling_bool = False
duplicated_time = 64
guidance_scals2= 10
description = 'noresampling'
 
for seed in seed_list:
    print(f'***********************seed{seed}*************************')
    print(f"Running ours method for {args.task} with seed {seed}")
    
    res = mango_eval(
        task,
        task_name=args.task,
        forwardmodel=forwardmodel,
        inverse_model=inverse_model,
        logging_dir=args.logging_dir,
        hidden_size=args.hidden_size,
        num_samples=args.budget,
        device=args.device,
        seed=seed,
        clip_dic=clip_dic,
        augment=augment,
        gen_condition=y_given,
        guidance=True,
        num_steps=num_steps,
        inference_scaling_bool=inference_scaling_bool,
        duplicated_time=duplicated_time,
        guidance_scals2=guidance_scals2,
        is_discrete=task.is_discrete,
        description=description,
    )
 
    y_unob_max = task.oracle.internal_dataset.y.max()
    y_unob_min = task.oracle.internal_dataset.y.min()

    designs = res['x']
    preds =   res['y_gen']
    # preds = res['y']
    scores = res['y_scores']
 
    print("Input structure:\n The number of the designs is: ", designs.shape[1], "; the dimension of the designs is: ", designs.shape[-1])
    print("Output structure:\n The number of the predictions is: ", preds.shape[1], "; the dimension of the predictions is: ", preds.shape[-1])

    
    # task = design_bench.make(args.task)
    if task.is_normalized_y:
        best_training_score = task.denormalize_y(np.max(task.dataset.y))[0]
    else:
        best_training_score = np.max(task.dataset.y)
    print("Training dataset:\n The best score during training is: ",best_training_score )
    oracle = np.max(scores)
    print(f'Oracle results:\n On the given design: the Oracle is: {oracle}')
    print(f' The corresponding design is: {designs[np.argmax(scores)]}')

    k = 1
    percentile = 100
    one_shot = scores[np.argmax(preds)][0]
    normalize_one_shot = (one_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', one_shot, 'the abs gap is: ', np.abs(oracle - one_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_one_shot, 'the abs gap is: ', np.abs(oracle - one_shot))
    reported_results_1shot.append(normalize_one_shot)

    k = 4
    percentile = 100
    preds_flat = preds.flatten()
    sorted_indices = np.argsort(preds_flat)
    top_64_indices = sorted_indices[-k:][::-1]
    few_shot = np.percentile(scores[top_64_indices],percentile)
    indices = np.where(scores[top_64_indices] == few_shot)[0]
    normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    # print(f' The corresponding design is: {designs[top_64_indices][indices]}')
    reported_results_16shot.append(normalize_few_shot)


    k = 16
    percentile = 100
    preds_flat = preds.flatten()
    sorted_indices = np.argsort(preds_flat)
    top_64_indices = sorted_indices[-k:][::-1]
    few_shot = np.percentile(scores[top_64_indices],percentile)
    indices = np.where(scores[top_64_indices] == few_shot)[0]
    normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    # print(f' The corresponding design is: {designs[top_64_indices][indices]}')
    reported_results_16shot.append(normalize_few_shot)


    k = 128
    percentile = 100
    preds_flat = preds.flatten()
    sorted_indices = np.argsort(preds_flat)
    top_64_indices = sorted_indices[-k:][::-1]
    few_shot = np.percentile(scores[top_64_indices],percentile)
    indices = np.where(scores[top_64_indices] == few_shot)[0]
    normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    # print(f' The corresponding design is: {designs[top_64_indices][indices]}')
    reported_results_128shot.append(normalize_few_shot)

    k = 256
    percentile = 100
    preds_flat = preds.flatten()
    sorted_indices = np.argsort(preds_flat)
    top_64_indices = sorted_indices[-k:][::-1]
    few_shot = np.percentile(scores[top_64_indices],percentile)
    indices = np.where(scores[top_64_indices] == few_shot)[0]
    normalize_few_shot = (few_shot -  y_unob_min)/(y_unob_max - y_unob_min)  
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on oracle score is: ', few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    print(f'Evaluation results 3:\n {k}-shot {percentile}th percentile of the predictions based on normalized oracle score is: ', normalize_few_shot, 'the abs gap is: ', np.abs(oracle - few_shot))
    # print(f' The corresponding design is: {designs[top_64_indices][indices]}')
    reported_results_256shot.append(normalize_few_shot)
    
print(f'***********************OVERALL RESULTS*************************')
print(f"reported_results_1shot: {np.mean(reported_results_1shot):.4f} ± {np.std(reported_results_1shot):.4f}")
print(f"reported_results_16shot: {np.mean(reported_results_16shot):.4f} ± {np.std(reported_results_16shot):.4f}")
print(f"reported_results_128shot: {np.mean(reported_results_128shot):.4f} ± {np.std(reported_results_128shot):.4f}")
print(f"reported_results_256shot: {np.mean(reported_results_256shot):.4f} ± {np.std(reported_results_256shot):.4f}")

y_given: tensor(1.)
{'simple_clip': True, 'clip_min': tensor([-1.9459, -1.9459, -1.9459, -1.9459, -1.9459, -1.9459, -1.9459, -1.9459,
        -1.9459, -1.9459, -1.9459, -1.9459, -1.9459, -1.9459, -1.9459, -1.9459,
        -1.9459, -1.9459, -1.9459, -1.9459, -1.9459, -1.9459, -1.9459, -1.9459,
         0.0000]), 'clip_max': tensor([1.9459, 1.9459, 1.9459, 1.9459, 1.9459, 1.9459, 1.9459, 1.9459, 1.9459,
        1.9459, 1.9459, 1.9459, 1.9459, 1.9459, 1.9459, 1.9459, 1.9459, 1.9459,
        1.9459, 1.9459, 1.9459, 1.9459, 1.9459, 1.9459, 1.0000])}
***********************seed49*************************
Running ours method for TFBind8-Exact-v0 with seed 49
the first 5 (no rank) design (normalized_x=False, normalized_y=False) is:
 tensor([[ 0.0000e+00,  0.0000e+00, -1.0000e+00, -4.7438e-01, -7.8301e-01,
          1.3465e+00, -9.6860e-03,  1.8422e+00,  1.2517e-02, -1.3529e-02,
          3.3239e-02,  1.7861e+00,  2.1375e-02,  7.8331e-04,  1.8006e+00,
          1.7513e+00,  1.1114e-03, -5.4558e

/home/tzhouaq/anaconda3/envs/design-bench/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


Input structure:
 The number of the designs is:  8 ; the dimension of the designs is:  3
Output structure:
 The number of the predictions is:  1 ; the dimension of the predictions is:  1
Training dataset:
 The best score during training is:  0.43929616
Oracle results:
 On the given design: the Oracle is: 0.989691972732544
 The corresponding design is: [[-2.         -2.          0.        ]
 [-0.07194437 -1.8257428   1.346271  ]
 [-0.06588991  1.9427408  -0.02293667]
 [-1.9445091  -1.926715   -1.9120358 ]
 [ 0.02794046 -0.04338714  1.91321   ]
 [-1.8855201  -1.9367075  -1.9317126 ]
 [ 1.9220456  -0.11523225  0.01987285]
 [ 1.7618656   0.1437159   0.02904889]]
Evaluation results 3:
 1-shot 100th percentile of the predictions based on oracle score is:  0.54383993 the abs gap is:  0.44585204
Evaluation results 3:
 1-shot 100th percentile of the predictions based on normalized oracle score is:  0.54383993 the abs gap is:  0.44585204
Evaluation results 3:
 4-shot 100th percentile of the pred